---

# Extração de Dados da API

## Propósito do Script

Este script foi criado para extrair dados de projetos legislativos da cidade de São Paulo, Brasil, a partir de uma API pública. Ele recupera dados de diferentes tipos de projetos para um determinado ano, extrai informações relevantes e as salva em um arquivo Excel.

## Como usar

1. **Definir o ano**: No final do script, na chamada da função `main()`, você pode definir o ano dos projetos que deseja extrair. Por exemplo, para extrair projetos do ano de 2023, você chamaria a função como `main(2023)`.

2. **Executar o script**: Execute o script em um ambiente Python que tenha as bibliotecas `requests` e `pandas` instaladas. O script fará requisições à API, extrairá os dados e os salvará em um arquivo Excel.

3. **Verificar o arquivo de saída**: Após a execução bem-sucedida do script, um arquivo Excel será criado no mesmo diretório do script. O nome do arquivo será `comissoes_relatorio_{ano}.xlsx`, onde `{ano}` é o ano que você definiu na chamada da função `main()`.

## Detalhes do Script

O script é dividido em várias funções para torná-lo mais modular e fácil de entender:

- `get_data_from_api(tipo, ano=2023)`: Esta função faz uma requisição à API para obter dados de um determinado tipo de projeto para um determinado ano.

- `extract_data(data_projetos)`: Esta função extrai dados relevantes dos projetos retornados pela API.

- `get_projetos_por_ano(ano=2023)`: Esta função faz uma requisição à API para obter todos os projetos para um determinado ano.

- `main(ano=2023)`: Esta é a função principal que coordena a execução de todas as outras funções.

---


In [11]:
import requests
import pandas as pd

# Função para obter dados da API
def get_data_from_api(tipo, ano=2023):
    endpoint_projetos = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx/ProjetosReunioesDeComissaoJSON"
    params_projetos = {"ano": ano, "tipo": tipo}
    response_projetos = requests.get(endpoint_projetos, params=params_projetos)
    return response_projetos.json() if response_projetos.text.strip() else []

# Função para extrair dados dos projetos
def extract_data(data_projetos):
    data_total = []
    for projeto in data_projetos:
        if 'encaminhamentos' in projeto:
            for encaminhamento in projeto["encaminhamentos"]:
                if 'comissoes' in encaminhamento:
                    for comissao in encaminhamento["comissoes"]:
                        parecer = comissao.get("conclusao", "")
                        if not parecer:
                            parecer = "Parecer ainda não emitido"
                        data = {
                            "Comissão": comissao["nome"],
                            "Projeto/Requerimento": f"{projeto['tipo']} {projeto['numero']}/{projeto['ano']}",
                            "Relatoria": comissao["relator"],
                            "Parecer": parecer,
                            "Chave": projeto["chave"]
                        }
                        data_total.append(data)
    return data_total

# Função para obter projetos por ano
def get_projetos_por_ano(ano=2023):
    endpoint_projetos_por_ano = "https://splegisws.saopaulo.sp.leg.br/ws/ws2.asmx/ProjetosPorAnoJSON"
    params_projetos_por_ano = {"ano": ano}
    response_projetos_por_ano = requests.get(endpoint_projetos_por_ano, params=params_projetos_por_ano)
    try:
        data_projetos_por_ano = response_projetos_por_ano.json()
    except json.JSONDecodeError:
        print("A resposta da API para ProjetosPorAnoJSON não é um JSON válido.")
        data_projetos_por_ano = []
    return data_projetos_por_ano

# Função principal
def main(ano=2023):
    tipos = ["PL", "PDL", "PR", "PLO"]
    data_total = []
    for tipo in tipos:
        data_projetos = get_data_from_api(tipo, ano)
        data_total.extend(extract_data(data_projetos))
    df_total = pd.DataFrame(data_total)
    df_projetos_por_ano = pd.DataFrame(get_projetos_por_ano(ano))
    df_total = pd.merge(df_total, df_projetos_por_ano, left_on="Chave", right_on="chave", how="left")
    df_total = df_total[["Comissão", "Projeto/Requerimento", "Relatoria", "Parecer", "ementa"]]
    df_total.columns = ["Comissão", "Projeto/Requerimento", "Relatoria", "Parecer", "Ementa"]
    df_total.to_excel(f"comissoes_relatorio_{ano}.xlsx", index=False)
    print(f"Dados extraídos com sucesso e salvos em comissoes_relatorio_{ano}.xlsx")

if __name__ == "__main__":
    main()




Dados extraídos com sucesso e salvos em comissoes_relatorio_2023.xlsx
